In [2]:
# Libraries, functions and input file
import structuremap.utils
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score, annotate_proteins_with_idr_pattern, get_extended_flexible_pattern, get_proximity_pvals, perform_enrichment_analysis, evaluate_ptm_colocalization
import pandas as pd
import os
import numpy as np
structuremap.utils.set_logger()

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
def retrieve_annotated_alphafold(uniprot_ids, afold_dir):

    # define cif and pae dirs
    cif_dir = os.path.join(afold_dir, 'cif')
    pae_dir = os.path.join(afold_dir, 'pae')
    # download the cif files
    valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
        proteins=uniprot_ids,
        out_folder=cif_dir
    )
    # download the pae files
    valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
        proteins=uniprot_ids,
        out_folder=pae_dir
    )
    # read downloaded files anf format them
    alphafold_annotation = format_alphafold_data(
        directory = cif_dir,
        protein_ids = uniprot_ids,
    )
    # annotate with full-sphere PSE values
    full_sphere_exposure = annotate_accessibility(
        df = alphafold_annotation,
        max_dist = 24,
        max_angle = 180,
        error_dir = pae_dir
    )
    # merge parsed alphafold with pPSE values
    alphafold_accessibility = alphafold_annotation.merge(
        full_sphere_exposure, how ='left', on=['protein_id','AA','position']
    )
    # annotate with partial sphere exposure values
    part_sphere_exposure = annotate_accessibility(
        df=alphafold_annotation, 
        max_dist=12, 
        max_angle=70, 
        error_dir=pae_dir
    )
    # merge with the previous dataframe
    alphafold_accessibility = alphafold_accessibility.merge(
        part_sphere_exposure, how='left', on=['protein_id','AA','position']
    )
    # discretize high and low accessibility
    alphafold_accessibility['high_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae <= 5, 1, 0)
    alphafold_accessibility['low_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae > 5, 1, 0)
    # smooth full sphere exposure
    alphafold_accessibility_smooth = get_smooth_score(
        alphafold_accessibility, 
        np.array(['nAA_24_180_pae']), 
        [10]
    )
    # discretize into "intrinsically disorder regions"
    alphafold_accessibility_smooth['IDR'] = np.where(
        alphafold_accessibility_smooth['nAA_24_180_pae_smooth10'] <= 34.27, 1, 0
    )
    # anotate short IDRs as IDRs that occur between two large ordered regions
    alphafold_accessibility_smooth_pattern = annotate_proteins_with_idr_pattern(
        alphafold_accessibility_smooth,
        min_structured_length = 80, 
        max_unstructured_length = 20
    )
    # extend the IDRs by 5 residues on both sides to increase coverage of detected psites
    alphafold_accessibility_smooth_pattern_ext = get_extended_flexible_pattern(
        alphafold_accessibility_smooth_pattern, 
        ['flexible_pattern'], [5]
    )
    return(alphafold_accessibility_smooth_pattern_ext)

def download_and_annotate_structures(input_file, output_file, afold_dir, uniprot_id_column = "Protein ID"):
    # read site level df
    site_level_df = pd.read_csv(input_file)
    unique_proteins = site_level_df[uniprot_id_column].unique()
    # retrieve annotated alphafold data
    annotated_afold = retrieve_annotated_alphafold(unique_proteins, afold_dir)
    # write to output csv
    annotated_afold.to_csv(output_file, index=False)

In [4]:
# download data for human cell lines
input_file = "data/raw_data/hek_hela_nglyco.csv"
output_file = "data/structuremap_data/human_alphafold_annotated.csv"
afold_cache_dir = "/Users/martin/Desktop/large_datasets/structuremap_analysis/"
download_and_annotate_structures(input_file, output_file, afold_cache_dir)

100%|██████████| 1336/1336 [00:04<00:00, 332.40it/s]

2023-03-31 16:47:33> Valid proteins: 0
2023-03-31 16:47:33> Invalid proteins: 65
2023-03-31 16:47:33> Existing proteins: 1271


Valid proteins:  0
Invalid proteins:  65
Existing proteins:  1271


100%|██████████| 1336/1336 [00:03<00:00, 386.65it/s]

2023-03-31 16:47:37> Valid proteins: 0
2023-03-31 16:47:37> Invalid proteins: 65
2023-03-31 16:47:37> Existing proteins: 1271



100%|██████████| 1271/1271 [00:01<00:00, 1267.76it/s]


In [ ]:
# download data for human cell lines
input_file = "data/raw_data/mouse_brain_nglyco.csv"
output_file = "data/structuremap_data/mouse_alphafold_annotated.csv"
afold_cache_dir = "/Users/martin/Desktop/large_datasets/structuremap_analysis/"
download_and_annotate_structures(input_file, output_file, afold_cache_dir)